<h1>Stock Volume Changes to Price Movement Analysis</h1>
<h3>Matt Quinlan and Wes Brown<h3>

<h2>Problem Identification</h2>
In the stock market world, there are two large groups with different mindsets around how to pick and choose which investments to place their money in. 

There are those that believe in "Fundamental" analysis, which focuses on the overall value of the stock and, in general, are looking to hold the stock for a long period of time, so that they can watch the stock price rise to reflect the value that they see. They like to follow the mantra of "buy low, sell high".

Then there are those that practice "Technical" analysis. Technical analysis focuses much more on the day to day trends. Investors here are commonly known as day-traders since they may buy and sell the same stock on the same day. These day traders are looking for trends in the data related to a specific stock - is the stock price moving up in a pattern that they've seen before? These investors follow the matra of "buy when it's going up, sell when it's going down".

For our project, we would like focus on those practicing technical analysis, the day traders. We would like to create a tool that could be useful to those looking at the day-to-day trends of a stock and help them in making a decision on whether it is a good buy or not.

<h2>Goal Determination</h2>
In the world of stock analysis, there is almost an overwhelming amount of data available to anyone. For our analysis, we are wanting to focus on two key pieces of information - the trade volume and the stock price. 

We would like to use the volume data to determine if there is an abnormally large amount of trading happening for this particular stock. We are going to look at the volume in comparsion to its past volume and also in comparsion to the volume of the stock market as a whole.

Once we have information regarding the volume of the trades, we will then look at the stock price information. Is the stock price trending up or down? By how much is it trending up or down? Has it been trending up or down? How volitile is the stock price for this stock?

Combining the information around the volume and stock price, we will determine which category that stock fits into: Strong Buy, Buy, Weak Buy, Hold, Weak Sell, Sell, and Strong Sell.

In the below section, we will talk about where we will get the data, how we will use the data for the specific pieces of the analysis, and then build the model and analysis.

<h2>Core Analysis and Model Building</h2>
Where the actual analysis will occur.

<h3>Reading in the Data</h3>

In [41]:
import pandas as pd
import os

filepath = os.path.join(os.getcwd(), 'data', 'stocks-screener-10-02-2020.csv')
stock_data = pd.read_csv(filepath)

#There is a known line at the end of the file that contains bad values
stock_data.dropna(inplace=True)

#Set the index for the dataframe to be the stock symbol
stock_data.set_index("Symbol", inplace=True)

stock_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 526 entries, A to ZTS
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          526 non-null    object 
 1   Last          526 non-null    float64
 2   Prev High     526 non-null    float64
 3   Prev Low      526 non-null    float64
 4   1M High       526 non-null    float64
 5   1M Low        526 non-null    float64
 6   Volume        526 non-null    float64
 7   Prev Vol      526 non-null    float64
 8   1M Avg Vol    526 non-null    float64
 9   Earnings ttm  526 non-null    float64
 10  Book Value    526 non-null    float64
 11  Div Yield     526 non-null    object 
dtypes: float64(10), object(2)
memory usage: 53.4+ KB


<h3>Calculate Volume Data</h3>
In the following sections we will calculate values that we need in order to understand the changes in volume for each of the stocks in the data set.

<h4>Daily Volume Change</h4>
One of the values we will look at is the change in from the previous day. We will calculate this using the Prev Vol and Volume values which represent the previous days volume and the current days volume respectively.

In [42]:
stock_data["Daily Volume Change"] = (stock_data["Volume"] - stock_data["Prev Vol"]) / stock_data["Prev Vol"]
stock_data[["Volume", "Prev Vol","Daily Volume Change"]]

,Volume,Prev Vol,Daily Volume Change
Symbol,,,
A,712248.0,723500.0,-0.015552
AAL,111883203.0,48841902.0,1.290722
AAP,487318.0,649600.0,-0.249818
AAPL,144711906.0,116120398.0,0.246223
ABBV,5554326.0,8777300.0,-0.367194
...,...,...,...
ZBH,971631.0,1101900.0,-0.118222
ZBRA,354500.0,461300.0,-0.231520
ZION,3176100.0,1515200.0,1.096159


<h4>Entire Stock Market Volume Change</h4>
When looking at the volume data, we want to understand if this volume change is due to that stock specifically or if there is something going in to impact the stock market overall. As an example, when the Feds adjust the interest rates, there is usually a spike in activity within the stock market. We want to attempt to take these types of volume changes into account by comparing a specific stocks volume change to the volume change of the stock market overall.

In [44]:
overall_today_volume = stock_data["Volume"].sum()
overall_yesterday_volume = stock_data["Prev Vol"].sum()
overall_volume_change = (overall_today_volume - overall_yesterday_volume) / overall_yesterday_volume

stock_data["Adjusted Daily Vol Change"] = stock_data["Daily Volume Change"] - overall_volume_change

print("Overall Stock Market Change: {}".format(overall_volume_change))
stock_data[["Daily Volume Change","Adjusted Daily Vol Change"]]

Overall Stock Market Change: 0.019906402417855622


,Daily Volume Change,Adjusted Daily Vol Change
Symbol,,
A,-0.015552,-0.035459
AAL,1.290722,1.270815
AAP,-0.249818,-0.269725
AAPL,0.246223,0.226317
ABBV,-0.367194,-0.387101
...,...,...
ZBH,-0.118222,-0.138129
ZBRA,-0.231520,-0.251426
ZION,1.096159,1.076253


<h4>Daily Volume vs Monthly Average</h4>
Another factor we would like to take into account is the average volume for the stock over the past month. While there may be a large increase between yesterday and today's volume, today's volume may not he be the outlier i.e. yesterday's volume may have been dramatically lower than the average. To account for that, we will look at the daily volume in comparsion to the monthly average.

In [45]:
stock_data["Monthly Vol vs Daily Vol"] = (stock_data["Volume"] - stock_data["1M Avg Vol"]) / stock_data["Prev Vol"]
stock_data[["Volume","1M Avg Vol", "Monthly Vol vs Daily Vol"]]

,Volume,1M Avg Vol,Monthly Vol vs Daily Vol
Symbol,,,
A,712248.0,1248995.0,-0.741876
AAL,111883203.0,58522461.0,1.092520
AAP,487318.0,703938.0,-0.333467
AAPL,144711906.0,180715969.0,-0.310058
ABBV,5554326.0,7395576.0,-0.209774
...,...,...,...
ZBH,971631.0,986319.0,-0.013330
ZBRA,354500.0,365629.0,-0.024125
ZION,3176100.0,1707929.0,0.968962


<h2>Flask Application</h2>
Not sure if this will be a final section or not.